<a href="https://colab.research.google.com/github/rohan-5076/neural-style-transfer/blob/main/Generative_Text_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

def generate_text_gpt2(prompt, max_length=100):
    model_name = "gpt2"
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [4]:
prompt = "The future of artificial intelligence"
print("---- GPT-2 Output ----")
print(generate_text_gpt2(prompt))


---- GPT-2 Output ----


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The future of artificial intelligence is uncertain. There are many questions that need to be answered before artificial intelligence can be built into every home, and it's not clear that the future of artificial intelligence will allow us to do so.

Cognitive scientist Michael R. Cramer has written extensively about the challenges of artificial intelligence, and he makes a number of compelling predictions about how it will play out. He has been a leading advocate for the development of artificial intelligence and has made numerous predictions about what's


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample corpus
corpus = [
    "Artificial intelligence is transforming education",
    "Machine learning enables systems to learn from data",
    "Deep learning is a subset of machine learning",
    "Neural networks are inspired by the human brain",
    "AI will personalize learning experiences"
]


In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

X = input_sequences[:, :-1]
y = tf.keras.utils.to_categorical(input_sequences[:, -1], num_classes=total_words)


In [7]:
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_seq_len - 1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, verbose=0)


In [8]:
def generate_text_lstm(seed_text, next_words=10):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break
    return seed_text


In [14]:
print("\n---- LSTM Output ----")
print(generate_text_lstm("AI will"))



---- LSTM Output ----
AI will personalize learning experiences experiences of machine from learning data machine
